In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import os, shutil
from glob import glob
import math
import tqdm
import tensorflowjs as tfjs
import numpy
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import sys
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.keras as efn
import re
import youtube_dl
from flask import Flask, jsonify, request
import time
import pyautogui
import copy

app=Flask(__name__)
model=load_model(r'/home/kashish/Desktop/Kashish/Projects/FECD/model/NPDIv3fullofull.12-0.17.h5')
saved_url=""
url_queue=[]
tab_queue=[]
flag_queue=[]
print("Awaiting Video Player")

def url_changed(saved_url):
    wait.until(
        lambda driver: driver.current_url!=saved_url)
    return True;

def url_strip(url):
    if "http://" in url or "https://" in url:
        url = url.replace("https://", '').replace("http://", '').replace('\"', '')
    if "/" in url:
        url = url.split('/', 1)[0]
    return url

def predict():
    while len(url_queue):
        video_url=url_queue.pop(0)
        prediction_images=[]
        print("Video Player found")
        start = time.time()
        ydl_opts={'verbose':True}
        ydl=youtube_dl.YoutubeDL(ydl_opts)
        info_dict=ydl.extract_info(video_url, download=False)
        vid_len=info_dict['duration']
        formats=info_dict.get('formats',None)
        print("Obtaining frames")
        for f in formats:
            if f.get('format_note',None) == '144p':
                url = f.get('url',None)
                cap = cv2.VideoCapture(url)
                if not cap.isOpened():
                    print('video not opened')
                    exit(-1)
                x=0
                count=30
                while count<=vid_len*30:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    filename =r"/home/kashish/Desktop/Kashish/Projects/FECD/temp/shot"+str(x)+".png"
                    x+=1;
                    cv2.imwrite(filename.format(count), frame)
                    count+=(vid_len/15)*30
                    cap.set(1,count)
                    if cv2.waitKey(30)&0xFF == ord('q'):
                        break
                cap.release()

        print(str(x)+" frames obtained successfully")
        print("Running Prediction Model")

        images = glob(r"/home/kashish/Desktop/Kashish/Projects/FECD/temp/*.png")

        for i in range(len(images)):
            img = image.load_img(images[i], target_size=(224,224,3))
            img = numpy.array(img)
            img = img/255
            prediction_images.append(img)

        prediction_images=numpy.array(prediction_images)
        pred=model.predict(prediction_images)
        y_pred = numpy.argmax(pred, axis=-1)

        for i in range(0,len(y_pred)):
            if y_pred[i]==1 and pred[i][1]<=0.95:
                y_pred[i]=0
        confiporn=[]
        confinonporn=[]
        for i in range(0,len(y_pred)):
            if y_pred[i]==1:
                confiporn.append(pred[i][1])
            else:
                confinonporn.append(pred[i][0])
        confiporn=numpy.array(confiporn)
        confinonporn=numpy.array(confinonporn)

        if(numpy.sum(y_pred)>=math.ceil(10*len(prediction_images)/100)):
            prediction=1
            confidence=numpy.mean(confiporn)
            tab_queue.pop(0)
        else:
            prediction=0
            confidence=numpy.mean(confinonporn)
            cur_flag=tab_queue.pop(0)
            if(cur_flag not in flag_queue):
                flag_queue.append(cur_flag)
        confi = numpy.round(confidence*100)

        end=time.time()
        print("Predicted to be of type "+str(prediction)+" in "+str(round(end-start,3))+" seconds") 

        print("...for "+url)
        folder=r"/home/kashish/Desktop/Kashish/Projects/FECD/temp"
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

@app.route('/send_url', methods=['POST'])
def send_url():
    resp_json = request.get_data()
    params = resp_json.decode() 
    url=params.split(',', 1)[0]
    url=url.replace("url=",'')
    tabId=params.split(',', 1)[1]
    tabId=tabId.replace("tab=",'')
    parent_url=url_strip(url)
    print(tabId)
    if((parent_url=="www.youtube.com" and "watch" in url) or (parent_url=="hellporno.com" and "videos" in url)):
        print("New video player found")
        url_queue.append(url)
        tab_queue.append(tabId)
        if len(url_queue)==1:
            predict()
    else:
        return jsonify({'message': 'success!'}), 200
    
@app.route('/quit_url', methods=['POST'])
def quit_url():
    resp_json = request.get_data()
    params = resp_json.decode() 
    url=params.split(',', 1)[0]
    url=url.replace("url=",'')
    tabId=params.split(',', 1)[1]
    tabId=tabId.replace("tab=",'')
    print("Url closed: " +url)
    if tabId in flag_queue:
        flag_queue.remove(tabId)
    return jsonify({'message': 'quit success!'}), 200    

@app.route('/blur_player',methods=['GET'])
def blur_player():
    temp_queue=copy.deepcopy(flag_queue)
    flag_queue.clear()
    print(temp_queue)
    return jsonify(temp_queue)

app.run(host='127.0.0.1', port=5000)

Awaiting Video Player
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[debug] Encodings: locale UTF-8, fs utf-8, out UTF-8, pref UTF-8
[debug] Encodings: locale UTF-8, fs utf-8, out UTF-8, pref UTF-8
[debug] youtube-dl version 2021.04.17
[debug] youtube-dl version 2021.04.17
[debug] Python version 3.8.5 (CPython) - Linux-5.8.0-50-generic-x86_64-with-glibc2.29
[debug] Python version 3.8.5 (CPython) - Linux-5.8.0-50-generic-x86_64-with-glibc2.29


7070
New video player found
New video player found
Video Player found

Video Player found


[debug] exe versions: none
[debug] Proxy map: {}


[HellPorno] sweet-lesbians-are-having-a-wild-time-fisting-one-another: Downloading webpage


...

[]


[2021-05-01 14:58:42,875] ERROR in app: Exception on /send_url [POST]
Traceback (most recent call last):
  File "/home/kashish/.local/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/kashish/.local/lib/python3.8/site-packages/flask/app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "/home/kashish/.local/lib/python3.8/site-packages/flask/app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "/home/kashish/.local/lib/python3.8/site-packages/flask/app.py", line 2097, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [01/May/2021 14:58:42] "POST /send_url HTTP/1.1" 500 -


Predicted to be of type 0 in 15.596 seconds
...for https://r6---sn-gwpa-a3v6.googlevideo.com/videoplayback?expire=1619882913&ei=QB-NYO3nOsWR3LUPxZe_2AY&ip=2405%3A201%3A5003%3Ab03b%3A80cd%3Af8f7%3A49a4%3Abda7&id=o-ALlT3RxT-IvX6KslDK8oVA4ga5FTMB3ru4aFKsJYCZfz&itag=160&aitags=133%2C134%2C135%2C136%2C137%2C160%2C242%2C243%2C244%2C247%2C248%2C278%2C394%2C395%2C396%2C397%2C398%2C399&source=youtube&requiressl=yes&mh=BZ&mm=31%2C29&mn=sn-gwpa-a3v6%2Csn-qxa7sn7s&ms=au%2Crdu&mv=m&mvi=6&pl=47&initcwndbps=652500&vprv=1&mime=video%2Fmp4&ns=vjVCAbTYrWH2XE3Qy43DF6wF&gir=yes&clen=2321585&dur=203.800&lmt=1585370850903774&mt=1619861098&fvip=6&keepalive=yes&fexp=24001373%2C24007246&c=WEB&txp=5511222&n=0iD0WsI_C9bxGrXqUNd&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cns%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRAIgd667_4L09-QnAlvFhUIFV5vuawTbnxIUvLyo_r_Ak44CIEZ-00uhNyMe_Mlxg2HZtr5A2VKyMxCDFZzUkepC88pY&sig=AOq0QJ8wRgIhANlN-

127.0.0.1 - - [01/May/2021 14:58:59] "GET /blur_player HTTP/1.1" 200 -


['70']
